In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_format ='retina'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Import data

In [2]:
#btc options data
df=pd.read_csv('data/btc_data_bsm.csv', sep=",", index_col=0)
df = df.reset_index(drop=False)
#df.drop(columns=['Unnamed: 0', 'interest_rate'], inplace=True)

In [2]:
import requests
import pandas as pd

# Deribit API endpoint and parameters
base_url = 'https://www.deribit.com/api/v2/public'
instrument_name = 'BTC-29JUN23-30000-C'  # Replace with the specific instrument name
num_trades = 1000  # The maximum number of historical trades to fetch

# Fetch historical trades by instrument
response = requests.get(
    f'{base_url}/get_last_trades_by_instrument',
    params={
        'instrument_name': instrument_name,
        'count': num_trades
    }
)

# Parse the JSON response
trades = response.json()['result']['trades']

# Convert to a Pandas DataFrame for analysis
df = pd.DataFrame(trades)

df.head()

""
